# Models

### Imports

In [1]:
import tensorflow as tf
import yaml

### Load Model and Labels

In [8]:
model_folder = "ConvNextBase"  # "EfficientNetV2M"  # 

In [16]:
!"./model_download.sh" "$model_folder"

ConvNextBase already downloaded...


In [9]:
model = tf.keras.models.load_model(model_folder)

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 300, 300, 3)]     0         
                                                                 
 convnext_base (Functional)  (None, 1024)              87566464  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 species (Dense)             (None, 89)                91225     
                                                                 
Total params: 87,657,689
Trainable params: 87,657,689
Non-trainable params: 0
_________________________________________________________________


In [11]:
with open("labels.yaml") as f:
    labels = yaml.load(f, yaml.SafeLoader)

### Run Predictions on a single Image

In [12]:
def load_and_crop_image(metadata, image_shape=[300,300]):
    """
    Loads an image from metadata and crops it to it's bounding box
    :param metadata: the image metadata
    :return: the cropped image
    """

    img = tf.io.read_file(metadata['file'])
    img = tf.io.decode_image(img, channels=3)
    img = tf.cast(img, tf.float32)

    if len(metadata['bbox']) == 0:
        bbox = [0.0, 0.0, 1.0, 1.0]
    else:
        bbox = metadata['bbox']
        bbox = [bbox[1], bbox[0], bbox[1]+bbox[3], bbox[0]+bbox[2]]

    cropped_img = tf.image.crop_and_resize([img], [bbox], [0], image_shape, method='bilinear')

    return cropped_img

In [13]:
def predict_and_decode(img, model, labels, top_k = None):
    """
    Performs model predictions on a given cropped image and returns the decoded top predictions
    :param img: the cropped image to run predictions
    :param model: the model to perform predictions with
    :param labels: the labels to decode the predictions
    :param top_k: if given return the top k predictions instead of the top prediction only
    :return: the decoded top predictions
    """
    
    preds = model.predict(img)[0]
    
    if top_k is not None:
        top_k_preds = tf.math.top_k(preds, k=top_k)
        preds_decoded = [(labels[top_k_preds.indices.numpy()[t]],
                                   top_k_preds.values.numpy()[t]) for t in range(top_k)]
    else:
        preds_decoded = labels[tf.math.argmax(preds).numpy()]
    return preds_decoded

In [17]:
metadata = {
    "file": "../data/MOF/img/20211118/02/IMAG0080.JPG",
    "bbox": [0.0871, 0.4609, 0.3328, 0.4593],
    "conf": 0.949,
}
img = load_and_crop_image(metadata)

In [18]:
predict_and_decode(img, model, labels)

1/1 [==============================] - 0s 52ms/step


'Sus_scrofa'